In [1]:
import os
import datetime
import mlflow
from mlflow.tracking import MlflowClient

In [2]:
mlflow.set_tracking_uri("https://ml-platform-3b36c8fb4e4070.ml.msk.vkcs.cloud/")

In [3]:
client = MlflowClient()

### Experiments

In [4]:
client.list_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/1', experiment_id='1', lifecycle_stage='active', name='driver-accident', tags={}>]

In [5]:
exp = client.get_experiment_by_name('driver-accident')
exp

<Experiment: artifact_location='mlflow-artifacts:/1', experiment_id='1', lifecycle_stage='active', name='driver-accident', tags={}>

In [6]:
client.list_run_infos(exp.experiment_id)

[<RunInfo: artifact_uri='mlflow-artifacts:/1/1e5290dfceb94a0a80c44b5f22660148/artifacts', end_time=1666735355114, experiment_id='1', lifecycle_stage='active', run_id='1e5290dfceb94a0a80c44b5f22660148', run_uuid='1e5290dfceb94a0a80c44b5f22660148', start_time=1666735339712, status='FINISHED', user_id='jupyter-admin'>,
 <RunInfo: artifact_uri='mlflow-artifacts:/1/26b40e069ae34d82b35999f81671b606/artifacts', end_time=1666735337573, experiment_id='1', lifecycle_stage='active', run_id='26b40e069ae34d82b35999f81671b606', run_uuid='26b40e069ae34d82b35999f81671b606', start_time=1666735166368, status='FINISHED', user_id='jupyter-admin'>,
 <RunInfo: artifact_uri='mlflow-artifacts:/1/9e16fa18829f427fac6223bc96ba3ce2/artifacts', end_time=1666734870952, experiment_id='1', lifecycle_stage='active', run_id='9e16fa18829f427fac6223bc96ba3ce2', run_uuid='9e16fa18829f427fac6223bc96ba3ce2', start_time=1666734855576, status='FINISHED', user_id='jupyter-admin'>,
 <RunInfo: artifact_uri='mlflow-artifacts:/1/4

In [7]:
run_info = client.list_run_infos(exp.experiment_id)[0]
run_info

<RunInfo: artifact_uri='mlflow-artifacts:/1/1e5290dfceb94a0a80c44b5f22660148/artifacts', end_time=1666735355114, experiment_id='1', lifecycle_stage='active', run_id='1e5290dfceb94a0a80c44b5f22660148', run_uuid='1e5290dfceb94a0a80c44b5f22660148', start_time=1666735339712, status='FINISHED', user_id='jupyter-admin'>

In [8]:
run_id = run_info.run_id

In [9]:
run = client.get_run(run_id)
run

<Run: data=<RunData: metrics={'f1': 0.9106661532537544,
 'precision': 0.9080745341614906,
 'recall': 0.9132726075006247,
 'roc_auc': 0.9027537535789404}, params={'best_params': "{'depth': 10, 'l2_leaf_reg': 7, 'learning_rate': 0.03}",
 'categorial features': "['sex', 'car_class']",
 'features': "['age', 'sex', 'car_class', 'driving_experience', "
             "'speeding_penalties', 'parking_penalties', 'total_car_accident']",
 'model_type': "<class 'catboost.core.CatBoostClassifier'>",
 'param_grid': "{'learning_rate': [0.03, 0.1], 'depth': [4, 6, 10], "
               "'l2_leaf_reg': [1, 3, 5, 7, 9]}",
 'target': 'has_car_accident'}, tags={'mlflow.log-model.history': '[{"run_id": "1e5290dfceb94a0a80c44b5f22660148", '
                             '"artifact_path": "driver-accident", '
                             '"utc_time_created": "2022-10-25 '
                             '22:02:33.015435", "flavors": {"python_function": '
                             '{"loader_module": "mlflow.cat

In [10]:
run.data.metrics

{'roc_auc': 0.9027537535789404,
 'precision': 0.9080745341614906,
 'recall': 0.9132726075006247,
 'f1': 0.9106661532537544}

In [11]:
run.data.params

{'features': "['age', 'sex', 'car_class', 'driving_experience', 'speeding_penalties', 'parking_penalties', 'total_car_accident']",
 'categorial features': "['sex', 'car_class']",
 'target': 'has_car_accident',
 'model_type': "<class 'catboost.core.CatBoostClassifier'>",
 'param_grid': "{'learning_rate': [0.03, 0.1], 'depth': [4, 6, 10], 'l2_leaf_reg': [1, 3, 5, 7, 9]}",
 'best_params': "{'depth': 10, 'l2_leaf_reg': 7, 'learning_rate': 0.03}"}

### Models

In [12]:
last_models = client.list_registered_models()
reg_model = last_models[0]
reg_model

<RegisteredModel: creation_timestamp=1666731965169, description='', last_updated_timestamp=1666735355099, latest_versions=[<ModelVersion: creation_timestamp=1666735355099, current_stage='None', description='', last_updated_timestamp=1666735355099, name='driver-accident', run_id='1e5290dfceb94a0a80c44b5f22660148', run_link='', source='mlflow-artifacts:/1/1e5290dfceb94a0a80c44b5f22660148/artifacts/driver-accident', status='READY', status_message='', tags={}, user_id='', version='8'>], name='driver-accident', tags={}>

In [13]:
reg_model.latest_versions

[<ModelVersion: creation_timestamp=1666735355099, current_stage='None', description='', last_updated_timestamp=1666735355099, name='driver-accident', run_id='1e5290dfceb94a0a80c44b5f22660148', run_link='', source='mlflow-artifacts:/1/1e5290dfceb94a0a80c44b5f22660148/artifacts/driver-accident', status='READY', status_message='', tags={}, user_id='', version='8'>]

In [14]:
def get_last_prod_model(name):
    last_models = client.get_registered_model(name).latest_versions
    models = list(filter(lambda x: x.current_stage == 'Production', last_models))
    if len(models) == 0:
        return None
    else:
        return models[0]

In [17]:
model_version = get_last_prod_model('driver-accident')
model_version

<ModelVersion: creation_timestamp=1666735355099, current_stage='Production', description='', last_updated_timestamp=1666736149135, name='driver-accident', run_id='1e5290dfceb94a0a80c44b5f22660148', run_link='', source='mlflow-artifacts:/1/1e5290dfceb94a0a80c44b5f22660148/artifacts/driver-accident', status='READY', status_message='', tags={}, user_id='', version='8'>

In [18]:
model_version.version

'8'